# 👶 Baby Crib Monitoring Demo using VideoDB RTStream
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/integrations/twelvelabs/Baby_Crib_Monitoring_TwelveLabs.ipynb)
## 📖 Storytime: Why This Matters

Meet **Vidit** and **Meghna** — a young couple juggling demanding jobs and household responsibilities. After a long, exhausting day, all they hope for is a peaceful night's sleep. But their energetic little one has other plans.

Their child, once safely tucked into his crib, has recently discovered how to climb out. While the parents sleep, unaware, the baby risks injury by wandering unsupervised at night. How can they keep him safe without losing their much-needed rest?

---

## 🎥 Enter VideoDB RTStream

**VideoDB** offers the perfect solution for this problem. Using **RTStream**, we can let AI continuously monitor a live video feed, index scenes, detect specific events like **baby attempting to climb out of the crib**, and instantly send alerts to the parents when something risky happens.

In this demo, **Vidit and Meghna install an IP camera near the crib** and use **VideoDB RTStream** to power an AI monitoring system.  
As soon as the baby makes a move to climb out, AI detects it, triggers an event, and fires a real-time alert so the parents can step in.

---

## 🚀 What You'll Build in This Notebook

By the end of this notebook, you'll learn how to:
- Connect a live RTSP video stream to VideoDB
- Continuously analyze video scenes using AI-generated natural language descriptions
- Detect specific events like *"baby escaping crib"*
- Trigger real-time alerts on such events

So — **do you want to build an intelligent AI-powered baby monitor too?**  
Let's get started!

---
## 📦 Step 1: Install Dependencies  
Before setting up the AI-powered baby monitor, let’s install the necessary VideoDB SDK.


In [ ]:
!pip install -q videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Let's connect to VideoDB's API using your credentials to prepare for stream monitoring.

Please enter your `VIDEO_DB_API_KEY` in the input box that appears below after you run this cell.

Your input will be masked.


In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---

## 📦 Step 3: Connect to the Baby Crib RTSP Stream  
We’ll now connect to the live video stream of the crib using its RTSP URL.

In this demo, the stream is running at `rtsp://samples.rts.videodb.io:8554/crib`.




In [ ]:
rtsp_url = "rtsp://samples.rts.videodb.io:8554/crib"
crib_stream = coll.connect_rtstream(
    name="Baby Monitoring Stream",
    rtsp_url=rtsp_url,
)
print(crib_stream)

#### Let us list all the rtstreams in our collection.

In [ ]:
def list_rtstreams():
    for rtstream in coll.list_rtstreams():
        if rtstream.status == "connected":
          print(f"""RTStream:
          ID            : {rtstream.id}
          Name          : {rtstream.name}
          Collection ID : {rtstream.collection_id}
          Created At    : {rtstream.created_at}
          Sample Rate   : {rtstream.sample_rate}
          Status        : {rtstream.status}
          """)
          print("-" * 80)

list_rtstreams()

RTStream:
          ID            : rts-0198795b-7cc0-7ef2-ba74-756cf38f18af
          Name          : Baby Monitoring Stream
          Collection ID : c-8ef029d5-a926-4859-9cc1-818549b3cb16
          Created At    : 2025-08-05T08:31:35.616286
          Sample Rate   : 30
          Status        : connected
          
--------------------------------------------------------------------------------



#### If you have already connected the stream, run the below cell with the **rtstream id** to reconnect.

In [ ]:
crib_stream = coll.get_rtstream("rts-0198795b-7cc0-7ef2-ba74-756cf38f18af")

In [ ]:
# To stop the stream
crib_stream.stop()

In [ ]:
# To start the stream
crib_stream.start()

---
### 👀 Let us have a look at the crib stream


#### 📺 Helper Function: Display Video Stream

This cell contains a small utility function to help visualize the video streams with helpful information. You don't need to modify this code.

In [ ]:
# To display the stream with relevant information

from IPython.display import HTML
import re
from datetime import datetime
from videodb import play_stream

def display_stream(video_url, video_name="🎥 Camera Feed"):

    match = re.search(r'/(\d{16})-(\d{16})\.m3u8', video_url)
    if match:
        start_ts = int(match.group(1)) / 1e6
        end_ts = int(match.group(2)) / 1e6
        start_time = datetime.utcfromtimestamp(start_ts).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.utcfromtimestamp(end_ts).strftime('%Y-%m-%d %H:%M:%S')
        time_range = f"{start_time} → {end_time} UTC"
    else:
        time_range = "Time Unknown"

    video_player_html = play_stream(video_url)

    return HTML(f"""
    <div style="position:relative;width:640px;">
      {video_player_html._repr_html_() if hasattr(video_player_html, '_repr_html_') else video_player_html}
      <div style="position:absolute;top:10px;left:10px;background:rgba(0,0,0,0.6);color:#fff;padding:6px 12px;border-radius:4px;font-size:13px;font-family:sans-serif;">
        <strong>{video_name}</strong><br>{time_range}
      </div>
    </div>
    """)


#### 🔗 Get & Display Recent Stream

This cell uses the helper function above to fetch and display the last few minutes of the stream.

In [ ]:
# To get last few minutes stream link
import time

def fetch_stream(rtstream):

    now = int(time.time())
    start = int(now - (2 * 60))
    stream_url = rtstream.generate_stream(start, now)
    return stream_url

video_url = fetch_stream(crib_stream)

video_name = "👶 Baby Monitor · Crib Activity Feed"
display_stream(video_url, video_name)

---

## 📦 Step 4: Index Scenes with Descriptions  
Now, we'll create a real-time scene index that periodically analyzes the video and generates natural language descriptions of what’s happening in the crib.

The AI model will watch for activity such as the baby moving, sitting, or attempting to climb out.


In [ ]:
crib_scene_index = crib_stream.index_visuals(
    batch_config={
        "type": "time",
        "value": 10,
        "frame_count": 6,
    },
    prompt="Describe the activity of the baby kept inside a baby crib. Notice if baby climbs out, is stading up in the crib or attempts to escape.",
    name="Baby_Crib_Index",
    model_name="twelvelabs-pegasus-1.2"
)
crib_index_id = crib_scene_index.rtstream_index_id
print("Scene Index ID:", crib_index_id)

#### Let us list the scene indexes created on our rtstream.

In [ ]:
def list_rtstream_indexes(rtstream):
    # List live stream indexes
    rtstream_indexes = rtstream.list_scene_indexes()
    for rtstream_index in rtstream_indexes:

        print(f"""RTStreamSceneIndex:
            Index ID       : {rtstream_index.rtstream_index_id}
            RTStream ID    : {rtstream_index.rtstream_id}
            Name           : {rtstream_index.name}
            Status         : {rtstream_index.status}
            Config         : {rtstream_index.extraction_config}
            Prompt         : {rtstream_index.prompt}
        """)
        print("-" * 80)

list_rtstream_indexes(crib_stream)

RTStreamSceneIndex:
            Index ID       : 8797c7f17f6fe669
            RTStream ID    : rts-01980ec0-90f5-7532-a728-aac362e9c490
            Name           : Baby_Crib_Index
            Status         : stopped
            Config         : {'frame_count': '6', 'time': '10'}
            Prompt         : Describe the activity of the baby kept inside a baby crib. Notice if baby climbs out, is stading up in the crib or attempts to escape.
        
--------------------------------------------------------------------------------



#### If you have already created a scene index, run the below cell with your **scene index id** to reconnect.

In [ ]:
crib_scene_index = crib_stream.get_scene_index(crib_index_id)

In [ ]:
crib_scene_index.stop()

In [ ]:
# To start the index
crib_scene_index.start()

---
### Let us see the result of the scene indexing

In [ ]:
import time
from datetime import datetime
from zoneinfo import ZoneInfo

def _convert_to_ist(timestamp: float) -> str:
    """Convert UTC timestamp to IST (Asia/Kolkata) datetime string."""
    return (
        datetime.fromtimestamp(timestamp)
        .astimezone(ZoneInfo("Asia/Kolkata"))
        .strftime("%Y-%m-%d %H:%M:%S")
    )

def get_scenes(rtstream, index_id):
    # Print indexed scenes
    rtstream_scene_index = rtstream.get_scene_index(index_id)
    scenes = rtstream_scene_index.get_scenes(page_size=5)
    if scenes:
        for scene in scenes.get("scenes"):
            start = _convert_to_ist(scene["start"])
            end = _convert_to_ist(scene["end"])
            description = scene["description"]
            print(f"{start}-{end}: {description}")
            print("-" * 80)
    else:
        print("Scenes not found for given index.")

get_scenes(crib_stream , crib_index_id)

2025-07-15 21:14:53-2025-07-15 21:15:03: The baby is standing up in the crib and appears to be attempting to climb out. The baby's actions suggest an attempt to escape from the crib, as they are holding onto the side rail and looking over the edge. There is no indication that the baby successfully climbs out during the video.
--------------------------------------------------------------------------------
2025-07-15 21:14:42-2025-07-15 21:14:52: The baby, initially standing inside the crib, begins to climb over the side rail using its hands and knees. It continues climbing until it reaches the top edge of the crib's side rail. As the baby descends, it maintains a grip on the bars and eventually reaches the bottom. The activity appears exploratory and indicative of the baby's developing motor skills, without any indication of an attempt to escape.
--------------------------------------------------------------------------------
2025-07-15 21:14:31-2025-07-15 21:14:41: The baby, wearing a

---
## 📦 Step 5: Define an Event for Baby Escape  
We’ll now create an event in VideoDB to detect when the AI spots the baby attempting to climb out.


In [ ]:
event_id = conn.create_event(
    event_prompt="Detect if the baby is trying to escape, standing up or climbing out of the crib.",
    label="baby_escape",
)
print("Event ID:", event_id)


Event ID: fe674d6bf037997f


---
## 📦 Step 6: Attach an Alert for the Escape Event  
To complete our setup, we’ll link a real-time alert to this event, which will notify the parents instantly through a webhook.


In [ ]:
# Enter link to your webhook url where you want alerts to go. You can create one simply on pipedream.
webhook_url=""

if webhook_url:
  alert_id = crib_scene_index.create_alert(
      event_id,
      callback_url="http://localhost:5678/webhook-test/1b67e694-d26a-4aa4-854d-e0997f40f741"
  )
  print("Alert ID:", alert_id)
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

Alert ID: 4d430e35dc53e5f9


---
## 📡 Example Alert Received at the Webhook

Here’s a real-time alert sent by our AI monitoring system to the webhook URL when the baby was detected trying to climb out of the crib:

---

**📬 Incoming Alert Payload (body):**

```json
{
  "event_id": "event-3adc40d26d6fed0d",
  "label": "baby_escape",
  "confidence": 0.95,
  "explanation": "The baby is actively trying to climb out of the crib by holding onto the top rail and attempting to pull itself up, which indicates an escape attempt.",
  "timestamp": "2025-05-28T23:36:39.979133+00:00",
  "start_time": "2025-05-29T05:06:36.612197+05:30",
  "end_time": "2025-05-29T05:06:46.612197+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019711a0-0fde-7911-b282-25bc0b4ecf65/1748475396000000-1748475407000000.m3u8"
}
```

---

### 📌 What This Tells Us:
- **event_id** — Unique ID for the event trigger  
- **label** — Type of event detected (`baby_escape`)  
- **confidence** — AI confidence score (0.95 = 95% sure)  
- **explanation** — AI-generated description of what’s happening  
- **timestamp** — When the alert was created  
- **start_time / end_time** — Time range of the detected scene  
- **stream_url** — This is a temporary link you can use to view the detected scene

✅ This confirms our AI-powered baby monitor is working perfectly — detecting escape attempts and instantly sending alerts.


---
### Let us have a look at the stream_url received in the alert.

In [ ]:
alert_stream_url = "https://rt.stream.videodb.io/manifests/rts-01980ec0-90f5-7532-a728-aac362e9c490/1752594293000000-1752594304000000.m3u8"
video_name = "🔔 Baby Monitor · baby_escape"

display_stream(alert_stream_url, video_name)

---
- Let us disable the alert after our task is done.

In [ ]:
crib_scene_index.disable_alert(alert_id)

- To enable the alert again

In [ ]:
crib_scene_index.enable_alert(alert_id)

- Now we can stop the stream

In [ ]:
crib_stream.stop()

NameError: name 'crib_stream' is not defined

---

## 🌙 Wrapping Up: Peace of Mind for Vidit and Meghna

In this notebook, we built a smart, AI-powered video monitoring system for a real-life problem faced by Vidit and Meghna.  

With this system in place, Vidit and Meghna can finally sleep peacefully, knowing their child is being safely monitored through AI-driven surveillance.  


---


**But this is just one story.**

- What if the same system could monitor an elderly parent at home — detecting falls or prolonged inactivity?  
- Or watch over a pet while the family is away, alerting them if it leaves a safe zone or behaves unusually?  
- Or even notify parents when a toddler approaches dangerous areas like staircases or kitchen counters?

The possibilities of real-time video intelligence at home are endless.

**What would *you* monitor next?**